# Downloading Sentinel-2 datasets

## AIM

The aim is to bulk download Sentinel-2 imagery from https://scihub.copernicus.eu/, covering Flanders (Belgium), using a set of search parameters.

## WORKFLOW

In [1]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt

import collections
import os
import logging

# import gui tkinter
from tkinter import Tk
import tkinter.filedialog as tkfd
import tkinter.simpledialog as tksd

In [2]:
logging.basicConfig(format='%(message)s', level='INFO')

### Login

A **tkinter GUI** is loaded to define:
- your Copernicus Open Access Hub **username**, and
- your Copernicus Open Access Hub **password**.

In [3]:
# tkinter GUI to ask username and password
root = Tk()

username = tksd.askstring(title = "USERNAME", 
                          prompt = "Log in with Copernicus Open Access Hub username ...")

password = tksd.askstring(title = "PASSWORD", 
                          prompt = "... and password") 

root.destroy()

In [4]:
api = SentinelAPI(username, password, "https://scihub.copernicus.eu/dhus")

### Set download directory and search area

A **tkinter GUI** is loaded:
- to define the **download directory**, and
- to locate a GeoJSON file, defining the **bouding box of Flanders**.

The tkinter input is printed to the console.

In [5]:
# tkinter GUI
root = Tk()
download_dir = tkfd.askdirectory(title = "Select download folder", initialdir = "C:/")
bounding_box_flanders = tkfd.askopenfilename(title = "Select input GeoJSON, defining the bounding box of Flanders", 
                                             initialdir = "C:/", 
                                             filetypes = (("GeoJSON", "*.geojson"),("all files", "*.*")))
root.destroy()

# print statements: the tkinter input is printed to the console
print("The selected download folder is:")
print("-", download_dir)
print(" ")
print("The selected bounding box is located at:")
print("-", bounding_box_flanders)

The selected download folder is:
- C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets
 
The selected bounding box is located at:
- C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson


### Define footprint

In [8]:
# define footprint = search the data by polygon
footprint_flanders_geojson = geojson_to_wkt(read_geojson(bounding_box_flanders))
print("Footprint loaded, bounding box:")
print(footprint_flanders_geojson)

Footprint loaded, bounding box:
POLYGON ((2.5406630 50.6879597, 2.5406630 51.5056400, 5.9098982 51.5056400, 5.9098982 50.6879597, 2.5406630 50.6879597))


### Set search parameters

A **tkinter GUI** is loaded to set 4 search parameters for querying the data:
- startdate (yyyymmdd),
- enddate (yyyymmdd),
- minimum cloudcover percentage (value from 0 to 100), 
- maximun cloudcover percentage (value from 0 to 100).

The tkinter input is printed to the console.

In [9]:
# tkinter GUI to define the parameters for querying the data
root = Tk()

startdate = tksd.askstring(title = "STARTDATE", 
                           prompt = "define startdate query (format: yyyymmdd)")

enddate = tksd.askstring(title = "ENDDATE", 
                         prompt = "define enddate query (format: yyyymmdd)")

min_cloud = tksd.askstring(title = "minimum cloudcover percentage", 
                           prompt = "define minimum cloudcover percentage (value from 0 to 100)")

max_cloud = tksd.askstring(title = "maximum cloudcover percentage", 
                           prompt = "define maximum cloudcover percentage (value from 0 to 100)")

root.destroy()

# print statements: tkinter input is printed to the console
print("The following parameters are selected:")
print("- startdate =", startdate)
print("- enddate =", enddate)
print("- minimum cloudcover percentage =", min_cloud)
print("- maximum cloudcover percentage =", max_cloud)

The following parameters are selected:
- startdate = 20160616
- enddate = 20160630
- minimum cloudcover percentage = 0
- maximum cloudcover percentage = 30


### Query Sentinel-2 data

Query the Sentinel-2 data, using:
- the predefined footprint (i.e. bounding box Flanders), and
- the predefined parameters.

The results are stored in a dictionary. A brief summary is printed to the console.

In [15]:
# Query the Sentinel-2 data by predefined footprint and parameters (date, cloudcover).
# Results of the query are stored in a dictionary.
products = api.query(footprint_flanders_geojson, 
                     startdate, 
                     enddate, 
                     platformname = 'Sentinel-2', 
                     cloudcoverpercentage = '[' + min_cloud + ' TO ' + max_cloud + ']'
                    )

# A very brief summary is printed to the console. 
products_count = 0
products_filenames = []
for k, v in products.items():
    filename = v['filename']
    filename = filename[:-5] + '.zip'
    products_count = products_count + 1
    products_filenames.append(filename)
print(products_count, "datasets are found:")
for product_filename in products_filenames:
    print("-", product_filename)

1 datasets are found:
- S2A_OPER_PRD_MSIL1C_PDMC_20160624T173545_R108_V20160624T103023_20160624T103023.zip


### Dictionary check

The results of the query are checked for what already exists in the donwload folder. A summary is printed to the console.

In [16]:
print("The download directory is checked for already downloaded datasets...")
no_action = 0
action = 0
filesize = 0
for k, v in products.items():
    filename = v['filename']
    size = v['size']
    size = float(size[:-3])
    zipfile = filename[:-5] + '.zip'
    folder = download_dir + "/" + zipfile
    if os.path.exists(folder):
        no_action = no_action + 1
        print(zipfile, "already exists!")
    else:
        action = action + 1
        filesize = filesize + size
        print(zipfile, "will be downloaded (", size, "GB )...")
print(" ")
print("TO SUMMARISE:")
print("---")
print(no_action, "file(s) already exist(s). The file(s) will be checked and updated if necessary.")
print(action, "file(s), or a total of", filesize, "GB, will be downloaded.")

The download directory is checked for already downloaded datasets...
S2A_OPER_PRD_MSIL1C_PDMC_20160624T173545_R108_V20160624T103023_20160624T103023.zip already exists!
 
TO SUMMARISE:
---
1 file(s) already exist(s). The file(s) will be checked and updated if necessary.
0 file(s), or a total of 0 GB, will be downloaded.


### Download

All new datasets are downloaded, existing datasets are checked and updated if necessary.

In [17]:
api.download_all(products, directory_path = download_dir, checksum = True, check_existing = True)

Will download 1 products
MD5 checksumming: 100%|███████████████████| 7.13G/7.13G [01:52<00:00, 63.4MB/s]
C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets\S2A_OPER_PRD_MSIL1C_PDMC_20160624T173545_R108_V20160624T103023_20160624T103023.zip was already downloaded.
1/1 products downloaded


(OrderedDict([('fecc8b23-334f-432f-b1e8-7d6f5dbcb819',
               {'date': datetime.datetime(2016, 6, 24, 10, 30, 23),
                'downloaded_bytes': 0,
                'footprint': 'POLYGON((5.768514805040419 49.61961011420237,5.76603610582172 49.57468117064941,5.763545147601364 49.53174440621896,5.763667108760206 49.53174069023494,5.71408665774242 48.63304460468163,7.200980618578432 48.5883487810417,7.206138543130609 48.64878448309814,7.708337912131061 48.657376206712975,9.066139888020086 48.66460174366902,10.4242909875692 48.65581480408966,11.847478451946776 48.62982157570405,11.901972660129278 49.57135020997286,11.956634167396428 50.46945531986655,12.017025008525813 51.41234265506691,10.579515067442992 51.43967797551486,10.579544021469632 51.44054116582309,10.50920791123296 51.4410149183699,10.438876794404743 51.44235231685392,10.438850408836755 51.441488815004725,9.140455380717382 51.45023422258088,9.140457957691327 51.451098078258774,9.070084949391758 51.45070820629918,8

In [18]:
print("Downloading complete! The downloads can be found at", download_dir)